In [1]:
# -*- coding: utf-8 -*-
import os
import numpy as np
import csv
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.utils.data

import torch
from torch.autograd import Variable
from torch.nn.parameter import Parameter
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.init as init
file1= 'ptbdb_normal.csv'
file2= 'ptbdb_abnormal.csv'
file3= 'mitbih_test.csv'
file4= 'mitbih_train.csv'

torch.manual_seed(19)

dfnorm= pd.read_csv(file1, header=None,dtype=np.float32)
dfabnorm= pd.read_csv(file2, header=None,dtype=np.float32)
dftrain = pd.read_csv(file4, header=None,dtype=np.float32)
dftest = pd.read_csv(file3, header=None,dtype=np.float32) 

npnorm=dfnorm.values
npabnorm=dfabnorm.values


nptrain=dftrain.values
nptest=dftest.values

TrainTor=torch.from_numpy(nptrain).double()
TestTor=torch.from_numpy(nptest).double()
valsize=int(TestTor.size()[0]*0.8)
testsize=TestTor.size()[0]-valsize
ValTor, TestTor=torch.utils.data.random_split(TestTor,(valsize,testsize) )



abnormTor=torch.from_numpy(npabnorm).double()
normTor= torch.from_numpy(npnorm).double()




In [2]:
def outsize(insize,filtersize,stride,padding):
    x=float((insize-filtersize+2*padding))/stride +1
    return int(x)

bsize=int(4)
filter1=20
stride1=1
padding1=0
filter2=5
lin1=outsize(187,filter1,stride1,padding1)
lin2=25
lin3=25
out=5



In [3]:
trainloader = torch.utils.data.DataLoader(TrainTor, batch_size=bsize,
                                          shuffle=True, num_workers=2,drop_last=True)

valloader = torch.utils.data.DataLoader(ValTor, batch_size=bsize,
                                         shuffle=False, num_workers=2,drop_last=True)

testloader = torch.utils.data.DataLoader(TestTor, batch_size=bsize,
                                         shuffle=False, num_workers=2,drop_last=True)

classes = ('0', '1', '2', '3',
           '4') 
#%%


In [4]:
import torch.nn as nn
import torch.nn.functional as F

out1= outsize(187,filter1,stride1,padding1)
out2= outsize(out1,filter2,2,padding1)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Conv1d(in_channels=1, out_channels=5, kernel_size=filter1, stride=stride1).double()
        self.conv2 = nn.Conv1d(in_channels=5, out_channels=10,kernel_size=filter2,stride=2).double()
        self.fc1 = nn.Linear(out2*10, lin2).double()
        self.fc3 = nn.Linear(lin2, out).double()
        #self.fc1 = nn.Linear(out2*10, lin2).double()
        #self.fc2 = nn.Linear(lin2, lin3).double()
        #self.fc3 = nn.Linear(lin3, out).double()

    def forward(self, x):
        x=x.view(bsize,1,187)
        x = F.relu(self.conv1(x))
        #x = F.relu(self.conv2(x))
        
        x=F.relu(self.conv2(x))
        
        x = x.view(-1,int(10*out2))
        x = F.relu(self.fc1(x))
        #x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [5]:
#%%
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9, weight_decay=0.005)

#%%
for epoch in range(5):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs=data[:,:187]
        labels = data[:,187].long()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')


[1,  2000] loss: 0.666
[1,  4000] loss: 0.491
[1,  6000] loss: 0.394
[1,  8000] loss: 0.375
[1, 10000] loss: 0.352
[1, 12000] loss: 0.317
[1, 14000] loss: 0.297
[1, 16000] loss: 0.287
[1, 18000] loss: 0.261
[1, 20000] loss: 0.248
[2,  2000] loss: 0.248
[2,  4000] loss: 0.235
[2,  6000] loss: 0.228
[2,  8000] loss: 0.225
[2, 10000] loss: 0.216
[2, 12000] loss: 0.219
[2, 14000] loss: 0.219
[2, 16000] loss: 0.200
[2, 18000] loss: 0.206
[2, 20000] loss: 0.206
[3,  2000] loss: 0.205
[3,  4000] loss: 0.198
[3,  6000] loss: 0.179
[3,  8000] loss: 0.192
[3, 10000] loss: 0.192
[3, 12000] loss: 0.188
[3, 14000] loss: 0.202
[3, 16000] loss: 0.185
[3, 18000] loss: 0.178
[3, 20000] loss: 0.170
[4,  2000] loss: 0.176
[4,  4000] loss: 0.178
[4,  6000] loss: 0.180
[4,  8000] loss: 0.176
[4, 10000] loss: 0.171
[4, 12000] loss: 0.177
[4, 14000] loss: 0.178
[4, 16000] loss: 0.181
[4, 18000] loss: 0.165
[4, 20000] loss: 0.176
[5,  2000] loss: 0.179
[5,  4000] loss: 0.152
[5,  6000] loss: 0.164
[5,  8000] 

In [6]:
#Validation:
for epoch in range(1):  # loop over the dataset multiple times

    running_loss = 0.0
    acc=0.0
    for i, data in enumerate(valloader, 0):
        # get the inputs
        inputs=data[:,:187]
        labels = data[:,187].long()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        acc+=float(sum(torch.max(net(inputs),1)[1]==labels))/labels.size()[0]
        running_loss += loss.item()
        if i % 500 == 499:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 500))
            print('[%d, %5d] accuracy: %.3f' %
                  (epoch + 1, i + 1, acc / 500))
            acc=0.0
            running_loss = 0.0



[1,   500] loss: 0.145
[1,   500] accuracy: 0.964
[1,  1000] loss: 0.163
[1,  1000] accuracy: 0.956
[1,  1500] loss: 0.166
[1,  1500] accuracy: 0.955
[1,  2000] loss: 0.150
[1,  2000] accuracy: 0.960
[1,  2500] loss: 0.162
[1,  2500] accuracy: 0.961
[1,  3000] loss: 0.160
[1,  3000] accuracy: 0.958
[1,  3500] loss: 0.150
[1,  3500] accuracy: 0.963
[1,  4000] loss: 0.191
[1,  4000] accuracy: 0.945


In [7]:
del(net)
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? n
Nothing done.


In [ ]:
def outsize(insize,filtersize,stride,padding):
    x=float((insize-filtersize+2*padding))/stride +1
    return int(x)
bsize=int(4)
filter1=10
stride1=1
padding1=0

lin1=outsize(187,filter1,stride1,padding1)
lin2=120
lin3=84
out=5
outsize(187,filter1,stride1,padding1)

In [10]:
out1= outsize(187,filter1,stride1,padding1)
filter2=2
out2= outsize(out1,filter2,2,padding1)
print(out2)

91
